# Hindle (1990)

* **Statistic**: 
    * Subj/Obj-V Mutual Information
    * Subj/Ojb-V Similarity
    * Noun Similarity
* **Corpus**:
    * Brown (NLTK)
* **Parsing**:
    * Type: Dependency
    * Library: Spacy
* **Categorization**:
    * Subject
    * Object


### A. Math

* **Subj/Obj-V Mutual Information**

    * $C_{subj/obj}(n,v) = log_2\frac{\frac{f(n,v)}{N}}{\frac{f(n)}{N}\cdot\frac{f(v)}{N}}$, where $f(n,v)$ is the frequency of noun $n$ occurring as the subj/obj of verb $v$; $f(n)$ the frequency of the noun $n$ occurring as the argument of any verb, and $f(v)$ is the frequency of the verb $v$; $N$ is the count of clauses in the dataset.


* **Subj/Obj-V Similarity**

    * $ {SIM}_{subj/obj}(v_i,n_j,n_k) = \begin{cases} 
    min(C_{subj/obj}(v_i,n_j), C_{subj/obj}(v_i,n_k)), & \text{if } C_{subj/obj}(v_i,n_j)>0 \text{ and } C_{subj/obj(v_i,n_k)}(v_i,n_k)>0, \\
    abs(max(C_{subj/obj}(v_i,n_j), C_{subj/obj}(v_i,n_k))), & \text{if } C_{subj/obj}(v_i,n_j)<0 \text{ and } C_{subj/obj(v_i,n_k)}(v_i,n_k)<0, \\
    0, & \text{otherwise}
    \end{cases} $


* **Noun Similarity**

    * $ {SIM}(n_1,n_2) = \sum_{i=1}^V {SIM}_{subj}(v_i,n_1,n_2) + {SIM}_{obj}(v_i,n_1,n_2) $, where $V$ is the number of unique verbs.
    

**NB**: In practice, the so-called *noun* in Hindle (1990) includes more than 'NOUN'-tagged words. For instance, *it* and *much*, which are considered in the paper, are tagged 'PRON' and 'ADV' respectively in brown. Therefore, in actually counting, we extract all 'arguments' instead, which is defined as the constituents which are in either 'nsubj' or 'dobj' relationship with the main verb.

### B. Extract Obj-V & Subj-V Matrices

In [1]:
from nltk.corpus import brown
from spacy.en import English

In [2]:
import random
import numpy as np
from collections import Counter, defaultdict

In [3]:
from nltk.stem import PorterStemmer
porter = PorterStemmer()

In [4]:
def frequency_matrices():
    """
    FUNCTION:
        computes subj-V & obj-V frequency matrices with: 
          - rows: verbs
          - cols: subj/obj
        returns 
          - verb-to-index dictionary
          - subj-to-index dictionary
          - obj-to-index dictionary
          - subj-V frequency matrix
          - obj-V frequency matrix
          - size of dataset
          - argument counts
          - verb counts
    """
    # extract info from corpus
    sents = [' '.join(sent) for sent in brown.sents()]
    tagged_words = brown.tagged_words(tagset='universal')
    w2t = defaultdict(str,{w:t for w,t in tagged_words})
    
    # parser corpus
    parser = English()
    parsed_corpus = [parser(sent) for sent in sents]
    
    # find subj/obj-v pairs
    subj_v_pairs = []
    obj_v_pairs = []
    arguments = []
    verbs = []
    for sent in parsed_corpus:
        for token in sent:
            if w2t[token.head.orth_]=='VERB':
                arguments.append(token.orth_)
                if token.dep_=='nsubj':
                    subj_v_pairs.append((token.orth_,token.head.orth_))
                elif token.dep_=='dobj':
                    obj_v_pairs.append((token.orth_,token.head.orth_))
                else: pass
            elif w2t[token.orth_]=='VERB':
                verbs.append(token.orth_)
            else: continue
    
    # "standardize" subjs, objs and verbs by stemming
    subj_v_pairs = [(porter.stem(subj).lower(),porter.stem(v)) 
                     for subj,v in subj_v_pairs]
    obj_v_pairs = [(porter.stem(obj).lower(),porter.stem(v)) 
                    for obj,v in obj_v_pairs]
    arguments = [porter.stem(n).lower() for n in arguments]
    verbs = [porter.stem(v).lower() for v in verbs]
    
    # build pred/arg-to-index dictionaries
    verb_vocab = list({verb for _,verb in subj_v_pairs+obj_v_pairs})
    subj_vocab = list({subj for subj,_ in subj_v_pairs})
    obj_vocab = list({obj for obj,_ in obj_v_pairs})
    v2i = defaultdict(int, {v:i for i,v in enumerate(verb_vocab)})
    s2i = defaultdict(int, {s:i for i,s in enumerate(subj_vocab)})
    o2i = defaultdict(int, {o:i for i,o in enumerate(obj_vocab)})
    
    # build dependency matrix
    subj_v_matrix = np.zeros((len(subj_vocab),len(verb_vocab)))
    obj_v_matrix = np.zeros((len(obj_vocab),len(verb_vocab)))
    for s,v in subj_v_pairs:
        subj_v_matrix[s2i[s]][v2i[v]] += 1
    for o,v in obj_v_pairs:
        obj_v_matrix[o2i[o]][v2i[v]] += 1 
    
    return v2i, s2i, o2i, subj_v_matrix, obj_v_matrix, len(sents), Counter(arguments), Counter(verbs)


In [5]:
%%time
v2i, s2i, o2i, subj_v_matrix, obj_v_matrix, N, argument_counts, verb_counts = frequency_matrices()

CPU times: user 1min 48s, sys: 1.81 s, total: 1min 49s
Wall time: 1min 50s


### C. Compute Noun Similarities

* Subj/Obj-V Mutual Information Matrices: $C_{subj/obj}(n,v)$,
* Subj/Ojb-V Similarity Tensors: ${SIM}_{subj/obj}(v,n_j,n_k)$,
* Noun Similarity Matrix: ${SIM}(n_j,n_k)$.

**NB**: For one-time computation, one may compute all the tensors and matrix, but will take a LONG time to compute on a large corpus, so in the demo here, we only compute the Mutual Information Matrices, and compute Noun Similarities "on the spot".

In [7]:
import cPickle
path = "/Users/jacobsw/Desktop/FALL_2016/LIN389C_RSCH_COMPLING/BAYESIAN/CODE/COMPUTED/"

In [8]:
from __future__ import division

In [9]:
log2 = lambda x: np.log2(x) if x!=0 else np.log2(1e-20)
div = lambda x,y: 0. if y==0 else x/y

In [10]:
def build_mi_matrices(v2i, s2i, o2i, s_v, o_v, 
                      N, arg_c, v_c):
    
    mi_subj = lambda n,v: log2( div( (s_v[s2i[n]][v2i[v]]/N), (arg_c[n]/N)*(v_c[v]/N) ) )
    mi_obj = lambda n,v: log2( div( (o_v[o2i[n]][v2i[v]]/N), (arg_c[n]/N)*(v_c[v]/N) ) )
    
    c_subj = np.zeros((len(s2i),len(v2i)))
    c_obj = np.zeros((len(o2i),len(v2i)))
    for v in v2i.keys():
        for s in s2i.keys():
            c_subj[s2i[s]][v2i[v]] = mi_subj(s,v)
        for o in o2i.keys():
            c_obj[o2i[o]][v2i[v]] = mi_obj(o,v)
    
    return c_subj, c_obj

In [11]:
%%time
c_subj, c_obj = build_mi_matrices(v2i, s2i, o2i, subj_v_matrix, obj_v_matrix, N, argument_counts, verb_counts)

CPU times: user 3min 34s, sys: 881 ms, total: 3min 35s
Wall time: 3min 35s


In [12]:
print 'eat-bread: ', c_obj[o2i['bread']][v2i['eat']]
print 'eat-chicken: ', c_obj[o2i['chicken']][v2i['eat']]
print 'eat-much: ', c_obj[o2i['much']][v2i['eat']]
print 'eat-it: ', c_obj[o2i['it']][v2i['eat']]

eat-bread:  8.46740428124
eat-chicken:  7.54986644144
eat-much:  3.53929719973
eat-it:  -0.22084602789


In [13]:
# IO

# with open(path+'mi_matrices.pickle', 'wb') as f:
#     cPickle.dump((c_subj,c_obj), f)
# with open(path+'mi_matrices.pickle', 'rb') as f:
#     c_subj,c_obj = cPickle.load(f)

In [14]:
def noun_sim(v2i, s2i, o2i, c_subj, c_obj, n_1, n_2):
    
    def sim_vnn(v, n_1, n_2, n2i, c_m): 
        # n2i: s2i or o2i
        # c_m: c_subj or c_obj
        c_vn_1 = c_m[n2i[n_1]][v2i[v]]
        c_vn_2 = c_m[n2i[n_2]][v2i[v]]
        if c_vn_1>0 and c_vn_2>0:
            return min(c_vn_1, c_vn_2)
        elif c_vn_1<0 and c_vn_2<0: 
            return abs(max(c_vn_1, c_vn_2))
        else:
            return 0
    
    return sum(sim_vnn(v,n_1,n_2,s2i,c_subj)+sim_vnn(v,n_1,n_2,o2i,c_obj) 
               for v in v2i.iterkeys()) / N 

# NB: the similarity score, as described in Hindle (1990), 
#     is not normalized by the size of the dataset (i.e. N),  
#     however since the score is in positive correlation with the
#     size of the corpus, it makes sense to normalize it.

In [15]:
from operator import itemgetter

In [16]:
def find_k_similar(arg, k, mode='sim'):
    # mode: sim or dissim
    # assuming v2i, s2i, o2i, c_subj, c_obj
    args = s2i.keys()+o2i.keys()
    sim = [(arg_i,noun_sim(v2i,s2i,o2i,c_subj,c_obj,arg,arg_i)) for arg_i in args]
    sim.sort(key=itemgetter(1),reverse=True)
    return sim[:k] if mode=='sim' else sim[-k:]

In [17]:
%%time
find_k_similar('boat',10, mode='sim')

CPU times: user 3min 25s, sys: 2.01 s, total: 3min 27s
Wall time: 3min 26s


[(u'boat', 9.0157302624390283),
 (u'boat', 9.0157302624390283),
 (u'lie', 9.0134328801042809),
 (u'lie', 9.0134328801042809),
 (u'consumm', 9.0134138960425663),
 (u'consumm', 9.0134138960425663),
 (u'samo', 9.013388596750465),
 (u'magnif', 9.013388596750465),
 (u'spacecraft', 9.013388596750465),
 (u'samo', 9.013388596750465)]

In [18]:
%%time
find_k_similar('boat',10, mode='dissim')

CPU times: user 4min 59s, sys: 3.12 s, total: 5min 2s
Wall time: 5min 1s


[(u'they', 8.2263245083546899),
 (u'they', 8.2263245083546899),
 (u'which', 8.205074035718523),
 (u'which', 8.205074035718523),
 (u'that', 8.20115843012535),
 (u'that', 8.20115843012535),
 (u'it', 7.8154963171502496),
 (u'it', 7.8154963171502496),
 (u'he', 7.7851723220975719),
 (u'he', 7.7851723220975719)]